In [1]:
import rioxarray
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import pydeck as pdk
import rasterio
import numpy as np
import pyproj
import shapely
import gc

print(pyproj.datadir.get_data_dir())

%matplotlib inline

/usr/local/Caskroom/miniconda/base/share/proj


To determine what part to get, we use the figure from [mines](https://eogdata.mines.edu/products/vnl/#monthly),

![coverage](https://eogdata.mines.edu/products/assets/img/tile_map.png)

As can be seen in the file [download directory](https://eogdata.mines.edu/nighttime_light/monthly/v10/2021/202109/vcmcfg/) the latitude and longtitude is switched around in some cases.

In [2]:
from pyalanysis.data import ViirsDnbMonthlyDataLoader, ViirsDnbMonthlyType
import os

vdl = ViirsDnbMonthlyDataLoader()



In [3]:
import tempfile
import requests
import os
import time

source_url = ("https://service.pdok.nl/cbs/pc4/atom/v1_0/downloads/cbs_pc4_2018.gpkg.zip")
dst_fn = "cbs_pc4_2018.gpkg.zip"


def download_file(url: str, dest_filename: str) -> str: 
    req = requests.get(url)
    file = open(dest_filename, 'wb')
    for chunk in req.iter_content(100000):
        file.write(chunk)
    file.close()

    return dest_filename

zip_fp = download_file(source_url, os.path.join(tempfile.mkdtemp(), dst_fn))

pc4 = gpd.read_file("zip://" + zip_fp + "!cbs_pc4_2018.gpkg")
pc4.head

<bound method NDFrame.head of       postcode  aantal_inwoners  aantal_mannen  aantal_vrouwen  \
0         1119               10         -99997          -99997   
1         1454              420            215             205   
2         1505             6470           3195            3275   
3         1623             2560           1215            1345   
4         1965             6095           3035            3060   
...        ...              ...            ...             ...   
4061      9995              685            355             330   
4062      9996               65             40              25   
4063      9997              615            325             290   
4064      9998              180             95              85   
4065      9999              100             50              50   

      aantal_inwoners_0_tot_15_jaar  aantal_inwoners_15_tot_25_jaar  \
0                            -99997                          -99997   
1                                70

In [4]:
gc.get_stats()

[{'collections': 666, 'collected': 10450, 'uncollectable': 0},
 {'collections': 62, 'collected': 1359, 'uncollectable': 0},
 {'collections': 5, 'collected': 361, 'uncollectable': 0}]

In [5]:
pc4 = pc4[["postcode", "geometry"]]
pc4.head

<bound method NDFrame.head of       postcode                                           geometry
0         1119  MULTIPOLYGON (((111425.535 476579.197, 111425....
1         1454  MULTIPOLYGON (((125231.079 492632.030, 125241....
2         1505  MULTIPOLYGON (((117982.293 494466.387, 117990....
3         1623  MULTIPOLYGON (((134217.968 517351.197, 134217....
4         1965  MULTIPOLYGON (((107022.752 501219.659, 107022....
...        ...                                                ...
4061      9995  MULTIPOLYGON (((239883.279 600110.514, 240113....
4062      9996  MULTIPOLYGON (((242493.596 601149.976, 242505....
4063      9997  MULTIPOLYGON (((240755.585 602191.415, 240752....
4064      9998  MULTIPOLYGON (((237960.007 602041.398, 237962....
4065      9999  MULTIPOLYGON (((235725.834 598999.629, 235737....

[4066 rows x 2 columns]>

In [6]:
gc.get_stats()

[{'collections': 667, 'collected': 10631, 'uncollectable': 0},
 {'collections': 62, 'collected': 1359, 'uncollectable': 0},
 {'collections': 5, 'collected': 361, 'uncollectable': 0}]

In [12]:
pc4_zh = pd.DataFrame({"postcode": [1428, 2159, 2161, 2162, 2163, 2171, 2172, 2181, 2182, 2191, 2201, 2202, 2203, 2204, 2211, 2212, 2215, 2216, 2221, 2222, 2223, 2224, 2225, 2231, 2232, 2235, 2241, 2242, 2243, 2244, 2245, 2251, 2252, 2253, 2254, 2261, 2262, 2263, 2264, 2265, 2266, 2267, 2271, 2272, 2273, 2274, 2275, 2281, 2282, 2283, 2284, 2285, 2286, 2287, 2288, 2289, 2291, 2292, 2295, 2311, 2312, 2313, 2314, 2315, 2316, 2317, 2318, 2321, 2322, 2323, 2324, 2331, 2332, 2333, 2334, 2341, 2342, 2343, 2351, 2352, 2353, 2355, 2361, 2362, 2371, 2374, 2375, 2376, 2377, 2381, 2382, 2391, 2394, 2396, 2401, 2402, 2403, 2404, 2405, 2406, 2407, 2408, 2409, 2411, 2412, 2415, 2421, 2431, 2432, 2435, 2441, 2445, 2451, 2461, 2465, 2471, 2481, 2491, 2492, 2493, 2495, 2496, 2497, 2498, 2511, 2512, 2513, 2514, 2515, 2516, 2517, 2518, 2521, 2522, 2523, 2524, 2525, 2526, 2531, 2532, 2533, 2541, 2542, 2543, 2544, 2545, 2546, 2547, 2548, 2551, 2552, 2553, 2554, 2555, 2561, 2562, 2563, 2564, 2565, 2566, 2571, 2572, 2573, 2574, 2581, 2582, 2583, 2584, 2585, 2586, 2587, 2591, 2592, 2593, 2594, 2595, 2596, 2597, 2611, 2612, 2613, 2614, 2616, 2622, 2623, 2624, 2625, 2626, 2627, 2628, 2629, 2631, 2632, 2635, 2636, 2641, 2642, 2643, 2645, 2651, 2652, 2661, 2662, 2665, 2671, 2672, 2673, 2675, 2676, 2678, 2681, 2684, 2685, 2691, 2692, 2693, 2694, 2711, 2712, 2713, 2715, 2716, 2717, 2718, 2719, 2721, 2722, 2723, 2724, 2725, 2726, 2727, 2728, 2729, 2731, 2735, 2741, 2742, 2743, 2751, 2752, 2761, 2771, 2801, 2802, 2803, 2804, 2805, 2806, 2807, 2808, 2809, 2811, 2821, 2825, 2831, 2841, 2851, 2855, 2861, 2865, 2871, 2872, 2901, 2902, 2903, 2904, 2905, 2906, 2907, 2908, 2909, 2911, 2912, 2913, 2914, 2921, 2922, 2923, 2924, 2925, 2926, 2931, 2935, 2941, 2951, 2952, 2953, 2954, 2957, 2959, 2961, 2964, 2965, 2967, 2968, 2969, 2971, 2973, 2974, 2975, 2977, 2981, 2982, 2983, 2984, 2985, 2986, 2987, 2988, 2989, 2991, 2992, 2993, 2994, 2995, 3011, 3012, 3013, 3014, 3015, 3016, 3021, 3022, 3023, 3024, 3025, 3026, 3027, 3028, 3029, 3031, 3032, 3033, 3034, 3035, 3036, 3037, 3038, 3039, 3041, 3042, 3043, 3044, 3045, 3046, 3047, 3051, 3052, 3053, 3054, 3055, 3056, 3059, 3061, 3062, 3063, 3064, 3065, 3066, 3067, 3068, 3069, 3071, 3072, 3073, 3074, 3075, 3076, 3077, 3078, 3079, 3081, 3082, 3083, 3084, 3085, 3086, 3087, 3088, 3089, 3111, 3112, 3113, 3114, 3115, 3116, 3117, 3118, 3119, 3121, 3122, 3123, 3124, 3125, 3131, 3132, 3133, 3134, 3135, 3136, 3137, 3138, 3141, 3142, 3143, 3144, 3145, 3146, 3147, 3151, 3155, 3161, 3162, 3165, 3171, 3172, 3176, 3181, 3191, 3192, 3193, 3194, 3195, 3196, 3197, 3198, 3199, 3201, 3202, 3203, 3204, 3205, 3206, 3207, 3208, 3209, 3211, 3212, 3214, 3216, 3218, 3221, 3222, 3223, 3224, 3225, 3227, 3231, 3232, 3233, 3234, 3235, 3237, 3238, 3241, 3243, 3244, 3245, 3247, 3248, 3249, 3251, 3252, 3253, 3255, 3256, 3257, 3258, 3261, 3262, 3263, 3264, 3265, 3267, 3271, 3273, 3274, 3281, 3284, 3286, 3291, 3292, 3293, 3295, 3297, 3299, 3311, 3312, 3313, 3314, 3315, 3316, 3317, 3318, 3319, 3328, 3329, 3331, 3332, 3333, 3334, 3335, 3336, 3341, 3342, 3343, 3344, 3351, 3352, 3353, 3354, 3355, 3356, 3361, 3362, 3363, 3364, 3366, 3371, 3372, 3373, 3381, 3465, 3466, 3651, 3652, 3653, 4201, 4202, 4203, 4204, 4205, 4206, 4207, 4208, 4209, 4213, 4221, 4223, 4225, 4241]})
pc4_zh = pc4.join(pc4_zh.set_index("postcode"), on="postcode",  how="inner", lsuffix='pc4')[["postcode", "geometry"]]
pc4_zh.head

<bound method NDFrame.head of       postcode                                           geometry
6         2406  MULTIPOLYGON (((105618.399 461540.262, 105613....
7         2512  MULTIPOLYGON (((80257.094 454582.660, 80258.20...
8         2542  MULTIPOLYGON (((79391.606 451102.503, 79383.97...
9         2585  MULTIPOLYGON (((82958.083 455081.114, 82958.07...
10        2636  MULTIPOLYGON (((86734.467 443702.531, 86739.50...
...        ...                                                ...
1401      4213  MULTIPOLYGON (((128678.055 427004.914, 128680....
1403      4221  MULTIPOLYGON (((127960.050 434929.345, 127977....
1404      4223  MULTIPOLYGON (((124529.959 433215.202, 124550....
1405      4225  MULTIPOLYGON (((124747.806 438109.714, 124748....
1409      4241  MULTIPOLYGON (((130139.618 432603.353, 130144....

[542 rows x 2 columns]>

In [13]:
gc.get_stats()

[{'collections': 683, 'collected': 12086, 'uncollectable': 0},
 {'collections': 63, 'collected': 1741, 'uncollectable': 0},
 {'collections': 7, 'collected': 873, 'uncollectable': 0}]

In [15]:
#viirs = vdl.open_viirs_monthly_file(vdl.get_viirs_dnb_monthly_file("75N060W", 2021, 12, ViirsDnbMonthlyType.STRAY_LIGHT_CORRECTED))

In [16]:
NETHERLAND_POLYGON = [
    {
        'type': 'Polygon',
        'coordinates': [[
            [3.2006621064250274, 53.58584446228139],
            [3.2006621064250274, 50.69456930499138],
            [7.276589840800027, 50.69456930499138],
            [7.276589840800027, 53.58584446228139]
        ]]
    }
]

In [17]:
#alan_nl = viirs[dict(band=0)].rio.clip(netherlands, crs=4326)
#alan_nl

In [18]:
pc4_zh = pc4_zh.to_crs("EPSG:4326")
pc4_zh = pc4_zh.set_index("postcode")
pc4_zh.head 

<bound method NDFrame.head of                                                    geometry
postcode                                                   
2406      MULTIPOLYGON (((4.66585 52.13985, 4.66579 52.1...
2512      MULTIPOLYGON (((4.29696 52.07447, 4.29698 52.0...
2542      MULTIPOLYGON (((4.28511 52.04308, 4.28500 52.0...
2585      MULTIPOLYGON (((4.33625 52.07931, 4.33625 52.0...
2636      MULTIPOLYGON (((4.39360 51.97753, 4.39367 51.9...
...                                                     ...
4213      MULTIPOLYGON (((5.00534 51.83102, 5.00538 51.8...
4221      MULTIPOLYGON (((4.99430 51.90221, 4.99456 51.9...
4223      MULTIPOLYGON (((4.94462 51.88663, 4.94492 51.8...
4225      MULTIPOLYGON (((4.94735 51.93063, 4.94736 51.9...
4241      MULTIPOLYGON (((5.02614 51.88141, 5.02621 51.8...

[542 rows x 1 columns]>

In [19]:
#alan_nl_df = alan_nl.drop_vars(["spatial_ref","band", "time"]).to_dataframe().reset_index()
#alan_nl_gdf = gpd.GeoDataFrame(alan_nl_df, geometry=gpd.points_from_xy(alan_nl_df.x, alan_nl_df.y), 
                               #crs=alan_nl.rio.crs.to_string())
#alan_nl_gdf = alan_nl_gdf.drop(["y", "x"], axis=1) 

In [20]:
#alan_southholland = alan_nl_gdf.sjoin(pc4_southholland, how="inner", predicate='intersects')
#alan_southholland.head() 

In [21]:
#alan_sh_pc4 = alan_southholland[['avg_rad9h', 'postcode']].groupby('postcode').agg({'avg_rad9h': np.mean})
#alan_sh_pc4.join(pc4_southholland.set_index("postcode"), on="postcode", how="inner").reset_index()



In [ ]:

x = alan_nl.time.item()
np.datetime64(x, "ns")

In [22]:
import tqdm
import time
import gc

NL_REGION = "75N060W"

def get_viirs_and_clip(area, year, month, clip_roi):
    retry = 0
    viirs = None
    while retry < 4 and not viirs:
        try:
            viirs = vdl.open_viirs_monthly_file(vdl.get_viirs_dnb_monthly_file(area, year, month, ViirsDnbMonthlyType.STRAY_LIGHT_CORRECTED))
        except:
            time.sleep(5)
            retry += 1
        else:
            break
    
    return viirs
            
            
    viirs_roi = viirs[dict(band=0)].rio.clip(clip_roi, crs=4326)
    return viirs_roi
    

def get_alan_for_dutch_zipcodes(year, month, postcode, pbar):
    nl_alan = get_viirs_and_clip(NL_REGION, year, month, NETHERLAND_POLYGON)
    pbar.update(1)
    
    timestamp = np.datetime64(nl_alan.time.item(), "ns")
    
    # Convert to geodataframe
    nl_alan = nl_alan.drop_vars(["spatial_ref","band", "time"]).to_dataframe().reset_index()
    nl_alan_gdf = gpd.GeoDataFrame(nl_alan, geometry=gpd.points_from_xy(nl_alan.x, nl_alan.y), 
                               crs=nl_alan.rio.crs.to_string())
    nl_alan_gdf = nl_alan_gdf.drop(["y", "x"], axis=1) 
    pbar.update(1)
    del(nl_alan)
    
    # Compute what points belong to what postcode in our region of interest
    roi_alan = nl_alan_gdf.sjoin(postcode, how="inner", predicate='intersects')
    del(nl_alan_gdf)
    pbar.update(1)
    
    roi_alan_pc4 = roi_alan[['avg_rad9h', 'postcode']].groupby('postcode').agg({'avg_rad9h': np.mean})
    roi_alan_pc4 = roi_alan_pc4.join(postcode.set_index("postcode"), on="postcode", how="inner").reset_index()
    roi_alan_pc4['time'] = timestamp
    pbar.update(1)
    
    return roi_alan_pc4

def get_annual_alan_for_dutch_zipcode(year, postcode):
    with tqdm.tqdm(total=12 * 4) as pbar:
        annual = get_alan_for_dutch_zipcodes(year, 1, postcode, pbar)

        for month in range(2, 13):
            df = get_alan_for_dutch_zipcodes(year, month, postcode, pbar)
            annual = pd.concat([annual, df])
            gc.collect()

    return annual

    

In [ ]:
south_holland_data = get_annual_alan_for_dutch_zipcode(2021, pc4_zh)
south_holland_data.shape

  2%|███▉                                                                                                                                                                                      | 1/48 [00:08<06:59,  8.92s/it]

In [ ]:
import os
os.environ["PYALANYSIS_MINES_USERNAME"] = "anton.bossenbroek@me.com"
os.environ["PYALANYSIS_MINES_PASSWORD"] = "cact-kah7phik2NOG"
